## Importing libraries

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as tkr
import numpy as np
import statsmodels.api as sm
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter('ignore')
from lifelines import KaplanMeierFitter

## Reading csv files

In [10]:
reads = pd.read_csv('../summary_data/summary_tcga_reads.csv')
vj = pd.read_csv('../summary_data/summary_tcga_VJ.csv')
cdr3 = pd.read_csv('../summary_data/summary_tcga_cdr3.csv')
microbiome = pd.read_csv('../summary_data/summary_tcga_microbiome.csv')
mtDNA = pd.read_csv('../summary_data/summary_tcga_mtDNA.csv')
purity = pd.read_csv('../summary_data/purity.csv')
purity = purity.dropna(subset=['CPE'])

## Joining data into single dataframe

In [11]:
sbt = cdr3.merge(vj, left_on='sample', right_on='sample', suffixes=('', 'vj'))
sbt = sbt.merge(reads, left_on='sample', right_on='sample', suffixes=(False, False))
sbt = sbt.merge(microbiome, left_on='sample', right_on='sample', suffixes=(False, False))
sbt = sbt.merge(mtDNA, left_on='sample', right_on='sample', suffixes=(False, False))
sbt['FileID'] = sbt['sample'].str[:36]

metadata = pd.read_csv('../summary_data/explore_case_table.tsv', delimiter="\t")
metadata = metadata.merge(pd.read_csv('../summary_data/gdc_sample_sheet.tsv', delimiter="\t"), left_on='Case ID', right_on='Case ID', suffixes=(False, False))
metadata['FileID'] = metadata['File Name'].str[:36]
metadata = metadata.loc[metadata['Age at diagnosis'] != "--"]
metadata = metadata.merge(purity, left_on=['Sample ID'], right_on=['Sample ID'], suffixes=(False, False))
data = metadata.merge(sbt, left_on=['FileID'], right_on=['FileID'], suffixes=(False, False))

l = data['Age at diagnosis'].str.split(" ", n = 4, expand = True) 
l.loc[((l[3] != "days") & (l[3] != "day")),2] = 0
data["Age"] = l[0].astype(float) + l[2].astype(float)/365
sbt = sbt.drop(columns=['n_unique_pe'])
sbt = sbt.drop(columns=['name'])
sbt = sbt.drop(columns=['mtDNA_ID'])
sbt = sbt.drop(columns=['FileID'])
data['Sex'] = data['Gender'] == "Female"

## Average of female samples

In [12]:
np.mean(data['Sex'])

0.5417515274949084

## Linear model for SBT features 

In [13]:
X = data[['Sex', 'Age','n_unique_pe','CPE']]
X['Sex'] = X['Sex'].astype(int)

X = np.array(X)
X = sm.add_constant(X)

pvalues = []
for i in range(1,len(sbt.columns)):
    y = data[sbt.columns[i]]
    y = np.array(y)
    mod = sm.OLS(y,X)
    fii = mod.fit()
    p_values = fii.summary2().tables[1]['P>|t|']
    pvalues.append([sbt.columns[i],p_values[1:]['x1'],p_values[1:]['x2'],p_values[1:]['x3'],p_values[1:]['x4']])

pvalues = pd.DataFrame(pvalues)   
pvalues.columns = ['feature','gender','age','#reads','purity']
pvalues.round(4)

,feature,gender,age,#reads,purity
0,nIGH,0.9087,0.9886,0.0002,0.0000
1,nIGK,0.1943,0.7990,0.0000,0.0000
2,nIGL,0.6644,0.8862,0.0000,0.0000
3,nTCRA,0.3361,0.1502,0.0000,0.0000
4,nTCRB,0.3024,0.2740,0.0000,0.0000
5,nTCRD,0.2581,0.6467,0.0000,0.0000
6,nTCRG,0.6051,0.2992,0.0000,0.0000
7,loadIGH,0.1598,0.5979,0.0026,0.0000
8,loadIGK,0.2989,0.9037,0.0000,0.0000
9,loadIGL,0.1478,0.8643,0.0004,0.0000
